In [8]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report
import torch
from UCF_crime_data import UCFCrimeDataset
from cnn import CnnNet
import torch.nn as nn


In [9]:
dataset = UCFCrimeDataset()

['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'NormalVideos', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']


In [10]:
net = CnnNet(64, len(dataset.classes),  c_kernels=[3, 3, 3, 3, 3, 3], in_channels=[3, 16, 32, 64, 86, 128, 128], out_channels=[16, 32, 64, 86, 128, 128])
criterion = nn.NLLLoss()
lr = 0.001
optimizer = torch.optim.AdamW(net.parameters(), lr=lr)
loss_test = []

ilość klas: 14
wielkość po warstawach conv: 10


In [11]:
if torch.cuda.is_available():
    net = net.cuda()
    net.cnn = net.cnn.cuda()
    print("GPU is available")
else:
    torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [12]:
def valid_classification(out, d):
    """
        Function calculating valid classification
        @ out - netowerk output
        @ d - destination value
        return: classification correctness in %
        """
    out = out.cpu().detach().numpy()
    d = d.cpu().detach().numpy()
    temp = abs(d - out)
    valid = sum(i < 0.5 for i in temp)
    return valid / temp.shape[0] * 100  # type:ignore


In [13]:
def test():
    loss_t = 0
    pk = []
    with torch.no_grad():
        for data, labels in dataset.testloader:
            labels = labels.cuda()
            data = data.cuda()
            out, _ = net(data)
            output = torch.argmax(out, dim=1)
            loss = criterion(out, labels)
            loss_t += loss.cpu().item()
            pk.append(valid_classification(output, labels))

    loss_test.append(loss_t)
    return np.average(pk)  # type:ignore


In [14]:
# sourcery skip: hoist-statement-from-loop
loss = 0
print("start")
for e in range(100):
    old_param = net.parameters
    # pass through all data
    for data, exp in dataset.trainloader:
        exp = exp.cuda()
        # pass data to cuda
        data = data.cuda()
        # clear gradient from previous epoch
        optimizer.zero_grad()
        out, sample = net(data)
        loss = criterion(out, exp)
        loss.backward()
        optimizer.step()

    pk = test()
    temp_lr: float = optimizer.param_groups[0]['lr']
    print("pk: {:.2f} %".format(pk))
    print("Learning rate: {:.10f}".format(temp_lr))
    print('Epoch: {}.............'.format(e), end=' ')
    print("Loss: {:.4f}".format(loss))
    
torch.save(net, "models/UCFCrime")


start


KeyboardInterrupt: 